In [11]:
import os
import glob
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType,DoubleType

spark = SparkSession.builder.appName("PF").config("spark.sql.caseSensitive", "True").getOrCreate()

In [12]:
#read inputs 
acc_meta = spark.read.options(inferSchema='True',header='True').csv('./data/other_input/account_metadata.csv')


#read master ledger file
df = spark.createDataFrame(pd.read_excel('./data/other_input/Master Ledger.xlsx',sheet_name="Master Ledger"))
df = df.withColumn("ID",col("ID").cast(IntegerType()))\
        .withColumn("Amount",col("Amount").cast(DoubleType()))\
        .withColumn("Subscriptions",col("Subscriptions").cast(BooleanType()))\
        .withColumn("Return",col("Return").cast(BooleanType()))\
        .withColumn("Real Amount",col("Real Amount").cast(DoubleType()))\
        .filter(col("Date") < current_date()-2)

df = df.withColumn("Date",to_date(col("Date"),"MM/dd/yyyy"))
df.printSchema()
df = df.dropna(how="all",subset= ["ID"])
df = df.replace('NaN',"")
df.orderBy("ID", ascending=False).show()

root
 |-- ID: integer (nullable = true)
 |-- Account: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Transaction Type: string (nullable = true)
 |-- Categories: string (nullable = true)
 |-- Categories 2: string (nullable = true)
 |-- Real Amount: double (nullable = true)
 |-- Note: string (nullable = true)
 |-- Subscriptions: boolean (nullable = true)
 |-- Return: boolean (nullable = true)
 |-- Account Type: string (nullable = true)
 |-- Owner: string (nullable = true)

+----+--------------------+--------------------+------+----------+----------------+-------------+------------+-----------+--------------------+-------------+------+------------+-----+
|  ID|             Account|                Item|Amount|      Date|Transaction Type|   Categories|Categories 2|Real Amount|                Note|Subscriptions|Return|Account Type|Owner|
+----+--------------------+--------------------+------+----------

In [13]:
#read all csv files exported from Empower
path = glob.glob('./data/empower_input/*.csv')
df2 = spark.read.options(inferSchema='True',header='True').csv(path)


#add more columns to df2
df2 = df2.withColumn("ID",lit(1))\
.withColumn("Item",col("Description")).drop("Description")\
.withColumn("Real Amount",col("Amount"))\
.withColumn("Amount",abs(col("Amount")))\
.withColumn("Account Type",lit(None))\
.withColumn("Owner",lit(None))\
.withColumn("Transaction Type",when(col("Real Amount") <0, "Expense").otherwise("Income"))\
.drop("Category")\
.withColumn("Owner",lit(None))\
.withColumn("Subscriptions",lit(False))\
.withColumn("Return",lit(False))\
.drop("Tags")\
.filter(col("Date") < current_date()-2)

df2.printSchema()

df2.show()

root
 |-- Date: date (nullable = true)
 |-- Account: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- ID: integer (nullable = false)
 |-- Item: string (nullable = true)
 |-- Real Amount: double (nullable = true)
 |-- Account Type: void (nullable = true)
 |-- Owner: void (nullable = true)
 |-- Transaction Type: string (nullable = false)
 |-- Subscriptions: boolean (nullable = false)
 |-- Return: boolean (nullable = false)

+----------+--------------------+------+---+--------------------+-----------+------------+-----+----------------+-------------+------+
|      Date|             Account|Amount| ID|                Item|Real Amount|Account Type|Owner|Transaction Type|Subscriptions|Return|
+----------+--------------------+------+---+--------------------+-----------+------------+-----+----------------+-------------+------+
|2023-04-18|Adv Plus Banking ...|1500.0|  1|               Zelle|    -1500.0|        null| null|         Expense|        false| false|
|2023-04-17|Adv

In [14]:
# union master ledger with empower, then 
max_date = df.select(max("Date")).first()[0]
df3 = df.unionByName(df2.filter(col("Date")> max_date), allowMissingColumns=True)
df3 = df3.drop("ID","Account Type","Owner").join(acc_meta, on = 'Account').orderBy("Date")
df3.orderBy("Date", ascending=False).show()

+--------------------+--------------------+------+----------+----------------+-------------+------------+-----------+--------------------+-------------+------+------------+-----+
|             Account|                Item|Amount|      Date|Transaction Type|   Categories|Categories 2|Real Amount|                Note|Subscriptions|Return|Account Type|Owner|
+--------------------+--------------------+------+----------+----------------+-------------+------------+-----------+--------------------+-------------+------+------------+-----+
|Discover It Ritu ...|             Tj Maxx| 217.2|2023-05-17|         Expense|         null|        null|     -217.2|                null|        false| false|      Credit| Ritu|
|Discover It Ritu ...|            Gulf Oil| 40.23|2023-05-17|         Expense|         null|        null|     -40.23|                null|        false| false|      Credit| Ritu|
|Amazon Prime Cred...|Automatic Payment...|322.17|2023-05-17|          Income|         null|        null|

In [15]:
df3.toPandas().to_csv("./data/output/out.csv", index_label="ID")